In [1]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext(appName="SERVER")
from pyspark.sql.session import SparkSession
spark = SparkSession(sc)

In [ ]:
from __future__ import print_function
from operator import add
import sys

from pyspark import SparkContext
from pyspark.streaming import StreamingContext

# Create a local StreamingContext with two working thread and batch interval of 1 second
#sc = SparkContext("local[2]", "NetworkWordCount")
sc.setLogLevel("ERROR")
ssc = StreamingContext(sc, 5)

# We need to create the checkpoint
ssc.checkpoint("checkpoint")

# Create a DStream that will connect to hostname:port, like localhost:9999
lines = ssc.socketTextStream("localhost", 10002)

print(lines.flatMap(lambda x: x.split(' ')) \
        .map(lambda word : (word, 1)) \
        .reduceByKey(add).pprint())

#print(lines.flatMap(lambda x: x.split(' ')) \
#        .map(lambda word : (word, 1)) \
#        .reduceByKeyAndWindow(add,30,10).pprint())


ssc.start()             # Start the computation
ssc.awaitTermination()  # Wait for the computation to terminate


None
-------------------------------------------
Time: 2021-03-25 19:34:20
-------------------------------------------

-------------------------------------------
Time: 2021-03-25 19:34:25
-------------------------------------------
('tired', 25)
('was', 26)
('very', 29)
('Alice', 20)

-------------------------------------------
Time: 2021-03-25 19:34:30
-------------------------------------------
('tired', 46)
('was', 52)
('very', 61)
('Alice', 41)

-------------------------------------------
Time: 2021-03-25 19:34:35
-------------------------------------------
('tired', 21)
('was', 26)
('very', 32)
('Alice', 21)

-------------------------------------------
Time: 2021-03-25 19:34:40
-------------------------------------------
('tired', 31)
('was', 25)
('very', 21)
('Alice', 23)

-------------------------------------------
Time: 2021-03-25 19:34:45
-------------------------------------------
('tired', 57)
('was', 44)
('very', 53)
('Alice', 46)

----------------------------------------

In [ ]:
from __future__ import print_function

import sys

from pyspark import SparkContext
from pyspark.streaming import StreamingContext

def updateFunc(new_values, last_sum):
        return sum(new_values) * (last_sum or 1)

# Create a local StreamingContext with two working thread and batch interval of 1 second
#sc = SparkContext("local[2]", "NetworkWordCount")
sc.setLogLevel("ERROR")
ssc = StreamingContext(sc, 5)

# We need to create the checkpoint
ssc.checkpoint("checkpoint")

# Create a DStream that will connect to hostname:port, like localhost:9999
lines = ssc.socketTextStream("localhost", 10002)


# Split each line into words
words = lines.flatMap(lambda line: line.split(" "))

# Count each word in each batch
pairs = words.map(lambda word: (word, 1))

#wordCounts = pairs.reduceByKey(lambda x, y: x + y)
wordCounts = pairs.updateStateByKey(updateFunc)

# Print the first ten elements of each RDD generated in this DStream to the console
wordCounts.pprint()

ssc.start()             # Start the computation
ssc.awaitTermination()  # Wait for the computation to terminate

